## Dependencies

In [1]:
!pip uninstall -y tensorflow numpy && pip install tensorflow-gpu==1.14.0 tf_sentencepiece

Uninstalling tensorflow-1.14.0:
  Successfully uninstalled tensorflow-1.14.0
Uninstalling numpy-1.16.5:
  Successfully uninstalled numpy-1.16.5
     |████████████████████████████████| 377.0MB 87kB/s 
     |████████████████████████████████| 2.7MB 31.0MB/s 
     |████████████████████████████████| 20.4MB 34.7MB/s 
ERROR: stable-baselines 2.2.1 requires tensorflow>=1.5.0, which is not installed.
ERROR: magenta 0.3.19 requires tensorflow>=1.12.0, which is not installed.
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [2]:
!curl -o many_to_one.zip https://dl.dropboxusercontent.com/s/xmroc5u6k17t5ds/many_to_one.zip?dl=0

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  113M  100  113M    0     0  33.4M      0  0:00:03  0:00:03 --:--:-- 33.3M


In [0]:
!unzip -q many_to_one.zip

In [4]:
!curl -o mlt.zip https://codeload.github.com/suyash/mlt/zip/master

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24327    0 24327    0     0  60066      0 --:--:-- --:--:-- --:--:-- 60066


In [0]:
!unzip -q mlt.zip && mv mlt-master/mlt ./

## Many to One Translation

In [0]:
import numpy as np
import tensorflow as tf
import tf_sentencepiece as tfs

from mlt.evaluation import predict
from mlt.layers import Attention, ConditionalNormalization, Gelu, MultiplyConstant, PaddingAndLookaheadMask, PaddingMask, PositionalEncoding

In [7]:
tf.__version__

'1.14.0'

In [0]:
sess = tf.InteractiveSession()

In [0]:
en_model_file = "final/sentencepiece/para_crawl/ende_plain_text/models/unigram/8192/a.model"
de_model_file = "final/sentencepiece/para_crawl/ende_plain_text/models/unigram/8192/b.model"
fr_model_file = "final/sentencepiece/para_crawl/enfr_plain_text/models/unigram/8192/b.model"
es_model_file = "final/sentencepiece/para_crawl/enes_plain_text/models/unigram/8192/b.model"
it_model_file = "final/sentencepiece/para_crawl/enit_plain_text/models/unigram/8192/b.model"

In [0]:
en_offset = 0
fr_offset = 8192
de_offset = fr_offset + 8192
es_offset = de_offset + 8192
it_offset = es_offset + 8192

In [11]:
with sess.as_default():
    model = tf.keras.experimental.load_from_saved_model("final/model114", custom_objects={
        "MultiplyConstant": MultiplyConstant,
        "PositionalEncoding": PositionalEncoding,
        "PaddingMask": PaddingMask,
        "PaddingAndLookaheadMask": PaddingAndLookaheadMask,
        "Attention": Attention,
        "ConditionalNormalization": ConditionalNormalization,
        "Gelu": Gelu,
    })

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### German to English

In [0]:
a = tfs.encode(["Hier finden Sie die Antworten, die Sie brauchen."], model_file=de_model_file, add_bos=True, add_eos=True)[0] + de_offset

In [13]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([1.0, 0.0, 0.0, 0.0]),
    tarf=tf.constant([1.0]),
    bos_id=en_offset + 1,
    eos_id=en_offset + 2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [16]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=en_model_file))], probs_[0]

(['Here you find the answers you need.',
  'Here you find the answers you need here.',
  'Click here to find the answers you need.',
  'Here you will find the answers you need.',
  'Here you can find the answers you need.'],
 array([0.700188  , 0.41780248, 0.37256515, 0.29034427, 0.26026225],
       dtype=float32))

### French to English

In [0]:
a = tfs.encode(["Vous pouvez trouver les réponses dont vous avez besoin ici."], model_file=fr_model_file, add_bos=True, add_eos=True)[0] + fr_offset

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    tarf=tf.constant([1.0]),
    bos_id=en_offset + 1,
    eos_id=en_offset + 2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [21]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=en_model_file))], probs_[0]

(['You can find the answers you need.',
  'You can find the answers you need here.',
  'You can find the answers that you need.',
  'You may find the answers you need.',
  'You can find here the answers you need.'],
 array([0.6454993 , 0.35628742, 0.2875612 , 0.23560844, 0.21897565],
       dtype=float32))

### Spanish to English

In [0]:
a = tfs.encode(["Puede encontrar las respuestas que necesita aquí."], model_file=es_model_file, add_bos=True, add_eos=True)[0] + es_offset

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 0.0, 1.0, 0.0]),
    tarf=tf.constant([1.0]),
    bos_id=en_offset + 1,
    eos_id=en_offset + 2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [26]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=en_model_file))], probs_[0]

(['You can find the answers you need here.',
  'You can find the answers you need.',
  'You can find the answers that you need here.',
  'You can find the answers that you need.',
  'You can find the answers we need here.'],
 array([0.77349657, 0.40411666, 0.39717805, 0.27532604, 0.23908341],
       dtype=float32))

### Italian to English

In [0]:
a = tfs.encode(["Puoi trovare le risposte che cerchi qui."], model_file=it_model_file, add_bos=True, add_eos=True)[0] + it_offset

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 0.0, 0.0, 1.0]),
    tarf=tf.constant([1.0]),
    bos_id=en_offset + 1,
    eos_id=en_offset + 2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [31]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=en_model_file))], probs_[0]

(['You can find the answers you want.',
  'You can find the courses you want here.',
  'You can find the answers you want here.',
  'You can find the stipulating answers.',
  'You can find the courses you want.'],
 array([0.4607614 , 0.26085025, 0.24746253, 0.22695974, 0.19222859],
       dtype=float32))

In [0]:
sess.close()